半月板障礙
半月板破裂

In [4]:

# finlab-format

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
from bs4 import BeautifulSoup
import json
import re
import requests
import os
from selenium.webdriver.chrome.options import Options
import random
from urllib.parse import urljoin


def roc_year_converter(dt: datetime):
    """
    doc: 將西元年轉換為民國年、月、日。
    """
    return dt.year - 1911, dt.month, dt.day


def sanitize_filename(title: str) -> str:
    """
    doc: 清理字串，使其可用作安全的檔案名稱。
    """
    s = re.sub(r'[\\/:*?"<>|]', '', title)
    s = s.replace(' ', '_')
    s = s.strip()
    if len(s) > 100:
        s = s[:100]
    return s


def parse_judgment_list(list_html: str, base_url: str):
    """
    doc: 解析判決列表頁面的HTML，提取所有判決的連結和標題。
    """
    print("步驟 6: 正在解析判決列表...")
    soup = BeautifulSoup(list_html, 'html.parser')
    judgments_info = []

    link_tags = soup.find_all('a', href=lambda href: href and 'data.aspx' in href)
    for tag in link_tags:
        full_link = urljoin(base_url, tag['href'])
        title = tag.text.strip()
        judgments_info.append({'link': full_link, 'title': title})

    print(f"解析完成！共找到 {len(judgments_info)} 筆判決連結。")
    return judgments_info


# --- 程式碼修改處 (1/3): 修改函式簽名，直接接收完整的檔案路徑 ---
def get_full_text(driver: webdriver.Chrome, judgment_url: str, judgment_title: str, full_md_path: str, full_pdf_path: str, proxy_address: str | None = None):
    """
    doc: 訪問單一判決連結，抓取內文，並直接儲存到指定的完整路徑。
    """
    try:
        driver.get(judgment_url)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        content_div = soup.find('div', class_='htmlcontent')
        if not content_div:
            print(f"警告：在 {judgment_url} 中未找到 'htmlcontent'。嘗試備用方案 'jud_content'...")
            content_div = soup.find('div', class_='jud_content')

        if not content_div:
            print(f"錯誤：在 {judgment_url} 中無法找到任何內文區塊。")
            full_text = "無法找到內文區塊。"
        else:
            full_text = content_div.get_text(separator='\n').strip()

        # 直接使用傳入的完整路徑儲存 Markdown 檔案
        with open(full_md_path, 'w', encoding='utf-8') as f:
            f.write(full_text)
        print(f"已儲存判決全文至: {full_md_path}")

        proxies = {'http': f'http://{proxy_address}', 'https': f'http://{proxy_address}'} if proxy_address else None
        pdf_link_tag = soup.find('a', id='hlExportPDF')
        if pdf_link_tag and pdf_link_tag.get('href'):
            pdf_url = urljoin(judgment_url, pdf_link_tag.get('href'))
            try:
                pdf_response = requests.get(pdf_url, stream=True, proxies=proxies, timeout=60)
                pdf_response.raise_for_status()
                # 直接使用傳入的完整路徑儲存 PDF 檔案
                with open(full_pdf_path, 'wb') as f:
                    for chunk in pdf_response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"已儲存判決PDF至: {full_pdf_path}")
            except requests.exceptions.RequestException as e:
                print(f"下載PDF失敗: {pdf_url}, 錯誤: {e}")
        else:
            print("未找到PDF下載連結。")
        return full_text
    except Exception as e:
        print(f"抓取內文時發生嚴重錯誤: {judgment_url}, 錯誤: {e}")
        return f"抓取失敗，錯誤: {e}"


def scrape_judicial_yuan_advanced(driver: webdriver.Chrome, keyword: str, years: int = 3):
    """
    doc: 執行司法院進階查詢。
    """
    url = 'https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx'
    print("步驟 1: 正在取得進階查詢頁面...")
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'jud_kw')))

    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * years)
    roc_start_y, roc_start_m, roc_start_d = roc_year_converter(start_date)
    roc_end_y, roc_end_m, roc_end_d = roc_year_converter(end_date)

    driver.find_element(By.ID, 'jud_kw').send_keys(keyword)
    driver.find_element(By.ID, 'dy1').send_keys(str(roc_start_y))
    driver.find_element(By.ID, 'dm1').send_keys(str(roc_start_m).zfill(2))
    driver.find_element(By.ID, 'dd1').send_keys(str(roc_start_d).zfill(2))
    driver.find_element(By.ID, 'dy2').send_keys(str(roc_end_y))
    driver.find_element(By.ID, 'dm2').send_keys(str(roc_end_m).zfill(2))
    driver.find_element(By.ID, 'dd2').send_keys(str(roc_end_d).zfill(2))

    civil_checkbox_xpath = '//input[@name="jud_sys" and @value="V"]'
    civil_checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, civil_checkbox_xpath)))
    if not civil_checkbox.is_selected():
        driver.execute_script("arguments[0].click();", civil_checkbox)
        print("已透過 JavaScript 點擊 '民事' 案件類別。")

    print(f"步驟 2: 準備使用關鍵字 '{keyword}' 查詢...")
    search_button = driver.find_element(By.ID, 'btnQry')
    search_button.click()
    print("步驟 3: 正在送出查詢請求...")

    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'iframe-data')))
        print("查詢成功！已取得結果頁面。")
    except Exception as e:
        print(f"等待結果頁面載入時發生錯誤或超時: {e}")
    return driver.page_source


if __name__ == '__main__':
    suggested_keywords = [
        "prp 韌帶", "prp 半月板", "半月板破裂", "半月板障礙",
        "膝部扭傷", "十字韌帶破裂", "半月板撕裂", "創傷性關節病變"
    ]
    search_years = 5
    
    # 基礎儲存目錄
    markdown_dir_base = "markdown_judgments"
    pdf_dir_base = "pdf_judgments"
    
    for search_keyword in suggested_keywords:
        print(f"\n{'='*25}\n[開始處理關鍵字]: {search_keyword}\n{'='*25}")
        
        # --- 程式碼修改處 (2/3): 為每個關鍵字建立專屬的子資料夾 ---
        sanitized_keyword = sanitize_filename(search_keyword)
        keyword_md_dir = os.path.join(markdown_dir_base, sanitized_keyword)
        keyword_pdf_dir = os.path.join(pdf_dir_base, sanitized_keyword)
        os.makedirs(keyword_md_dir, exist_ok=True)
        os.makedirs(keyword_pdf_dir, exist_ok=True)
        print(f"本關鍵字檔案將儲存於: {os.path.abspath(keyword_md_dir)}")

        driver = None
        try:
            chrome_options = Options()
            # chrome_options.add_argument('--headless')
            chrome_options.add_argument("--window-size=1920,1080")
            driver = webdriver.Chrome(options=chrome_options)
            driver.set_page_load_timeout(60)

            all_judgments_data_for_keyword = []
            page_count = 1

            search_page_html = scrape_judicial_yuan_advanced(driver, keyword=search_keyword, years=search_years)
            soup = BeautifulSoup(search_page_html, 'html.parser')
            iframe = soup.find('iframe', id='iframe-data')

            if not (iframe and iframe.get('src')):
                if "查無資料" in search_page_html:
                    print(f"關鍵字 '{search_keyword}' 查無資料，跳至下一個關鍵字。")
                else:
                    print(f"關鍵字 '{search_keyword}' 查詢後未找到 iframe src，跳至下一個關鍵字。")
                continue
            
            current_list_page_url = urljoin(driver.current_url, iframe.get('src'))
            print(f"步驟 4: 成功進入判決列表 iframe: {current_list_page_url}")
            time.sleep(random.uniform(3.0, 5.0))

            while True:
                print(f"\n--- 正在處理第 {page_count} 頁 ---")
                driver.get(current_list_page_url)
                try:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'data.aspx')]")))
                except Exception:
                    print(f"在第 {page_count} 頁等待判決連結超時，可能已是最後一頁。")
                    break

                list_html = driver.page_source
                judgments_info = parse_judgment_list(list_html, driver.current_url)
                if not judgments_info:
                    print(f"第 {page_count} 頁沒有找到判決連結，抓取結束。")
                    break

                for judgment in judgments_info:
                    sanitized_title = sanitize_filename(judgment['title'])
                    
                    # --- 程式碼修改處 (3/3): 實現編號與更穩健的重複檢查 ---
                    # 1. 檢查此標題的檔案是否已存在（不論編號為何）
                    is_duplicate = False
                    existing_files = os.listdir(keyword_md_dir)
                    for existing_file in existing_files:
                        # 如果有檔案的結尾是 "_標題.md"，就視為重複
                        if existing_file.endswith(f"_{sanitized_title}.md"):
                            is_duplicate = True
                            print(f"[跳過] 判決已存在: {existing_file}")
                            break
                    
                    if is_duplicate:
                        continue # 跳過此筆重複的判決

                    # 2. 如果不重複，計算新檔案的編號
                    # 編號等於目前資料夾中的檔案數量 + 1
                    new_file_number = len(existing_files) + 1
                    
                    # 3. 組合出包含編號的新檔名和完整路徑
                    base_filename = f"{new_file_number:02d}_{sanitized_title}"
                    full_md_path = os.path.join(keyword_md_dir, f"{base_filename}.md")
                    full_pdf_path = os.path.join(keyword_pdf_dir, f"{base_filename}.pdf")
                    
                    print(f"\n[抓取] 新判決: {judgment['title']}")
                    # 4. 呼叫函式，並傳入完整的檔案路徑
                    full_text = get_full_text(driver, judgment['link'], judgment['title'], full_md_path, full_pdf_path)
                    
                    all_judgments_data_for_keyword.append({
                        "關鍵字": search_keyword,
                        "檔案名": base_filename,
                        "連結": judgment['link'],
                        "標題": judgment['title'],
                        "內文": full_text
                    })
                    time.sleep(random.uniform(4.0, 8.0))

                list_soup = BeautifulSoup(list_html, 'html.parser')
                next_page_link_tag = list_soup.find('a', id='hlNext')
                if next_page_link_tag and next_page_link_tag.get('href'):
                    current_list_page_url = urljoin(current_list_page_url, next_page_link_tag.get('href'))
                    page_count += 1
                    print(f"\n找到下一頁，準備前往第 {page_count} 頁...")
                    time.sleep(random.uniform(5.0, 10.0))
                else:
                    print("沒有找到下一頁連結，此關鍵字所有頁面已抓取完畢。")
                    break
            
            if all_judgments_data_for_keyword:
                json_filename = f"judgments_data_{sanitized_keyword}.json"
                with open(json_filename, 'w', encoding='utf-8') as f:
                    json.dump(all_judgments_data_for_keyword, f, ensure_ascii=False, indent=4)
                print(f"\n已將關鍵字 '{search_keyword}' 的 {len(all_judgments_data_for_keyword)} 筆新資料儲存至 {json_filename}。")

        except Exception as e:
            import traceback
            print(f"處理關鍵字 '{search_keyword}' 時發生嚴重錯誤: {e}")
            traceback.print_exc()
        finally:
            if driver:
                driver.quit()
                print(f"已關閉瀏覽器，結束關鍵字 '{search_keyword}' 的處理。")

    print(f"\n{'='*25}\n所有關鍵字處理完畢！\n{'='*25}")



[開始處理關鍵字]: prp 韌帶
本關鍵字檔案將儲存於: c:\Users\user\Documents\GitHub\judgment_python\markdown_judgments\prp_韌帶
步驟 1: 正在取得進階查詢頁面...
已透過 JavaScript 點擊 '民事' 案件類別。
步驟 2: 準備使用關鍵字 'prp 韌帶' 查詢...
步驟 3: 正在送出查詢請求...
查詢成功！已取得結果頁面。
步驟 4: 成功進入判決列表 iframe: https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?ty=JUDBOOK&q=89936c00fe646240449ab5d6c8558b33

--- 正在處理第 1 頁 ---
步驟 6: 正在解析判決列表...
解析完成！共找到 20 筆判決連結。

[抓取] 新判決: 臺灣新竹地方法院 113 年度 訴 字第 3 號民事判決
已儲存判決全文至: markdown_judgments\prp_韌帶\01_臺灣新竹地方法院_113_年度_訴_字第_3_號民事判決.md
已儲存判決PDF至: pdf_judgments\prp_韌帶\01_臺灣新竹地方法院_113_年度_訴_字第_3_號民事判決.pdf

[抓取] 新判決: 臺灣臺南地方法院 114 年度 保險簡上 字第 1 號民事判決
已儲存判決全文至: markdown_judgments\prp_韌帶\02_臺灣臺南地方法院_114_年度_保險簡上_字第_1_號民事判決.md
已儲存判決PDF至: pdf_judgments\prp_韌帶\02_臺灣臺南地方法院_114_年度_保險簡上_字第_1_號民事判決.pdf

[抓取] 新判決: 臺灣彰化地方法院 113 年度 簡上 字第 182 號民事判決
已儲存判決全文至: markdown_judgments\prp_韌帶\03_臺灣彰化地方法院_113_年度_簡上_字第_182_號民事判決.md
已儲存判決PDF至: pdf_judgments\prp_韌帶\03_臺灣彰化地方法院_113_年度_簡上_字第_182_號民事判決.pdf

[抓取] 新判決: 臺灣桃園地方法院 114 年度 勞訴 字第 30 號民事判決
已儲存判決全文至